In [1]:
import os
import time
import json
from pathlib import Path

import torch
from datasets import load_dataset, load_metric
import sentencepiece as spm
from tokenizers import Tokenizer, trainers, models, pre_tokenizers, decoders, processors

# Additional imports for evaluation and logging
import sacrebleu
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
translation_dataset = load_dataset("wmt14", "de-en", split="train[:1%]")
sentiment_dataset = load_dataset("amazon_polarity", split="train[:1%]")  # Just an example in English, replace with multilingual.

In [3]:
#Phase 2: Experiments (Fine-tuning Models)

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM  # For translation tasks, e.g., MarianMT

def load_custom_tokenizer(tokenizer_type="bpe"):
    if tokenizer_type == "bpe":
        # Load a fast tokenizer from files
        # This is a simplified example, in practice you'd wrap it with PreTrainedTokenizerFast
        return AutoTokenizer.from_pretrained("tokenizers/bpe", use_fast=True)
    elif tokenizer_type == "sp":
        return AutoTokenizer.from_pretrained("tokenizers/sp_unigram_hf", use_fast=True)
    elif tokenizer_type == "wp":
        return AutoTokenizer.from_pretrained("tokenizers/wp", use_fast=True)
    else:
        raise ValueError("Unsupported tokenizer type")

# Example: Load baseline tokenizer (BPE)
baseline_tokenizer = load_custom_tokenizer("bpe")

In [4]:
# For translation tasks, you might use a MarianMT model or mBART, for sentiment XLM-R or mBERT.
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model_name = "Helsinki-NLP/opus-mt-en-de"  
baseline_model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

In [5]:
def preprocess_function_translation(examples):
    # Extract lists of source and target texts from the list of dictionaries
    source_texts = [item["en"] for item in examples["translation"]]
    target_texts = [item["de"] for item in examples["translation"]]
    
    # Tokenize the source texts
    inputs = baseline_tokenizer(
        source_texts,
        truncation=True,
        padding="max_length",
        max_length=128
    )
    
    # Tokenize the target texts
    with baseline_tokenizer.as_target_tokenizer():
        labels = baseline_tokenizer(
            target_texts,
            truncation=True,
            padding="max_length",
            max_length=128
        )
    
    inputs["labels"] = labels["input_ids"]
    return inputs

# Apply the preprocessing function
tokenized_translation_dataset = translation_dataset.map(preprocess_function_translation, batched=True)


In [6]:
from transformers import (
    TrainingArguments,
    Trainer,
)

train_args = TrainingArguments(
    output_dir="checkpoints/trans_bpe",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,  
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    fp16=torch.cuda.is_available()
)

trainer = Trainer(
    model=baseline_model,
    args=train_args,
    train_dataset=tokenized_translation_dataset,
    eval_dataset=tokenized_translation_dataset,  
)

trainer.train()

/Users/sg/anaconda3/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/56360 [00:00<?, ?it/s]

{'loss': 3.8996, 'grad_norm': 18.763490676879883, 'learning_rate': 4.9991128459900645e-05, 'epoch': 0.0}
{'loss': 2.5345, 'grad_norm': 3.4558730125427246, 'learning_rate': 4.998225691980128e-05, 'epoch': 0.0}
{'loss': 2.2961, 'grad_norm': 2.2678959369659424, 'learning_rate': 4.997338537970192e-05, 'epoch': 0.0}
{'loss': 1.9061, 'grad_norm': 2.102567195892334, 'learning_rate': 4.996451383960256e-05, 'epoch': 0.0}
{'loss': 2.0006, 'grad_norm': 1.8110806941986084, 'learning_rate': 4.9955642299503194e-05, 'epoch': 0.0}
{'loss': 1.6279, 'grad_norm': 7.101086616516113, 'learning_rate': 4.994677075940383e-05, 'epoch': 0.01}
{'loss': 1.7614, 'grad_norm': 1.3375777006149292, 'learning_rate': 4.993789921930447e-05, 'epoch': 0.01}
{'loss': 2.1774, 'grad_norm': 1.6551400423049927, 'learning_rate': 4.9929027679205115e-05, 'epoch': 0.01}
{'loss': 1.7076, 'grad_norm': 1.6849730014801025, 'learning_rate': 4.992015613910575e-05, 'epoch': 0.01}
{'loss': 1.8357, 'grad_norm': 2.656099319458008, 'learning_

  0%|          | 0/11272 [00:00<?, ?it/s]

{'eval_loss': 0.9538769125938416, 'eval_runtime': 994.2464, 'eval_samples_per_second': 45.349, 'eval_steps_per_second': 11.337, 'epoch': 1.0}


...

  0%|          | 0/11272 [00:00<?, ?it/s]

{'eval_loss': 0.5807990431785583, 'eval_runtime': 1111.7491, 'eval_samples_per_second': 40.556, 'eval_steps_per_second': 10.139, 'epoch': 5.0}


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


{'train_runtime': 26614.9996, 'train_samples_per_second': 8.47, 'train_steps_per_second': 2.118, 'train_loss': 0.87365803493516, 'epoch': 5.0}


TrainOutput(global_step=56360, training_loss=0.87365803493516, metrics={'train_runtime': 26614.9996, 'train_samples_per_second': 8.47, 'train_steps_per_second': 2.118, 'total_flos': 7642047389368320.0, 'train_loss': 0.87365803493516, 'epoch': 5.0})

In [7]:
seed = 123
sample = translation_dataset.shuffle(seed=seed).select(range(100))

def generate_predictions(model, tokenizer, dataset):
    model.eval()
    predictions = []
    references = []
    for example in dataset:
        input_ids = tokenizer(example["translation"]["en"], return_tensors="pt").input_ids.to(device)
        # Generate translations
        outputs = model.generate(input_ids)
        pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predictions.append(pred)
        references.append([example["translation"]["de"]])  # sacrebleu expects a list of references
    return predictions, references

preds, refs = generate_predictions(baseline_model, baseline_tokenizer, sample)
bleu_score = sacrebleu.corpus_bleu(preds, list(zip(*refs)))
print("BLEU Score:", bleu_score.score)

BLEU Score: 18.527955881408168


In [8]:
rare_word = "supercalifragilisticexpialidocious"
print("BPE Tokens:", baseline_tokenizer.tokenize(rare_word))

BPE Tokens: ['s', 'up', 'erc', 'al', 'if', 'rag', 'il', 'ist', 'ice', 'xp', 'ial', 'id', 'oc', 'ious']
